In [258]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 0. Intro
1. 计算得到GT1 GT2 GT3对于testing data的NLL值 or KL-D作为metric
2. 不包含InferNet

# 1. Preparations
## 1.1 global settings

In [259]:
from MLP.utils import *
import os
from tqdm import tqdm
from MLP.Config.config_base import BaseConfig
config = BaseConfig()

from importlib import reload
import metric_for_GTs_func

from metric_for_GTs_func import *
reload(metric_for_GTs_func)

<module 'metric_for_GTs_func' from 'D:\\Desktop\\PROJ\\PAProj\\data_handler\\cal_metric\\metric_for_GTs_func.py'>

In [260]:
ARTIFICIAL = True
noise_pct = 0.05           # 噪音占比:我们希望生成的data总体上最多浮动的百分比noise_pct

q=1                 # q=1表示不使用quantile 计算metric
CAL_GT1 = False    # 是否计算GT1，按理说只需要计算1次，得到file即可

if ARTIFICIAL:
        seed_list = [4,31,35,66,204,407,508,512]
else:
        seed_list = [3,31,62,204,223,407,508,626]
seed_list

[4, 31, 35, 66, 204, 407, 508, 512]

## 1.2 the data path


In [261]:
# Target data
if ARTIFICIAL:
        target_path = "../../data/artificial_targets_v2_" + "noise=" + str(noise_pct)
else:
        target_path = r"../../data/targets_all"

# target data是target_5时 TARGET = 5
TARGET = 1

# Output
output_path = r"../../MLP/metric_saved/"
prefix = ""
if ARTIFICIAL:
        prefix = "synthetic_"
else:
        prefix = "real_"


col_names_1 = ['GT1']
col_names_2 = ['GT2(Unified)', 'GT2(Avg)']
col_names_3 = ['GT3(Unified)', 'GT3(Avg)']


# For GT-2
# alpha = -0.013581112
# delta = 1
# labda = 3.312402533

MIN_LOSS = 1e-30


In [262]:
data_key = pd.read_csv(config.data_key_path,encoding="utf-8")

# target_path里有全部的target data地址
target_all_path = os.listdir(target_path)
len_all = len(target_all_path)
print(f"一共*{len(data_key)}*场auction")

assert len_all == len(data_key),"wrong in len_all"


一共*1276*场auction


# 2. GT1全局只需要算一次
1. 生成一个file

## 2.1 KL-D

In [224]:
if CAL_GT1:
        metric_GT1_KL = pd.DataFrame()

        for idx,file_i in enumerate(target_all_path):

                target_path_i = os.path.join(target_path, file_i)
                target_df = pd.read_csv(target_path_i,encoding="utf-8")
                target_df_uniq = pd.DataFrame(np.unique(target_df,axis=0)) # KL-D计算必须有这一步，去掉N值重复的sample

                # Get the auction setting of i_th according to the test_idx
                settings_df = data_key.iloc[idx,:]
                metric_GT1_KL.loc[idx,col_names_1[0]] = GT1_KL(settings_df,target_df_uniq)


In [225]:
if CAL_GT1:
        output_file = output_path + prefix +"GT1_metric_KL_all.csv"
        metric_GT1_KL.to_csv(output_file)
        output_file

## 2.2 NLL

In [226]:
if CAL_GT1:
        metric_GT1_NLL = pd.DataFrame()

        for idx,file_i in enumerate(target_all_path):

                target_path_i = os.path.join(target_path, file_i)
                target_df = pd.read_csv(target_path_i,encoding="utf-8")
                target = list(target_df.iloc[:,0])

                # Get the auction setting of i_th according to the test_idx
                settings_df = data_key.iloc[idx,:]
                metric_GT1_NLL.loc[idx,col_names_1[0]] = GT_1(settings_df,target)


In [227]:
if CAL_GT1:
        output_file = output_path + prefix +"GT1_metric_NLL_all.csv"
        metric_GT1_NLL.to_csv(output_file)
        output_file

# 3. GT2(Unified)和GT2(Avg)
- 不同于GT1，需要按照seed算
- 同时计算NLL和KL


In [118]:
metric_unified_KL = pd.DataFrame()
metric_unified_NLL = pd.DataFrame()

for seed in seed_list:
        setup_seed(seed)
        shuffled_indices = save_data_idx_simplified(len_all)
        train_idx,_,test_idx = get_data_idx(shuffled_indices, train_pct = config.train_pct, vali_pct=config.vali_pct)

        print(f"========== seed = {seed} ==========")

        if not ARTIFICIAL:
                params_unified_path = r"../../data/SA_PT/params_opitim_GT2_unified_seed="+str(seed)+".csv"
                params_avg_path = r"../../data/SA_PT/params_opitim_delta_GT2.csv"
        else:
                params_unified_path = r"../../data/SA_PT/params_artificial_GT2_unified_noise="+str(noise_pct)+ "_seed="+str(seed)+".csv"
                params_avg_path = r"../../data/SA_PT/params_opitim_artificial_GT2_noise_"+str(noise_pct)+".csv"

        # Unified
        params_unified = pd.read_csv(params_unified_path,encoding="utf-8")
        # Get params with the lowest avg_loss
        idx_param = params_unified.avg_loss.idxmin(0)
        params_unified = params_unified.iloc[idx_param,:]
        alpha_unified = params_unified.alpha
        labda_unified = params_unified.labda
        # print(f"params_unified:{params_unified}")

        # Avg
        params_avg = pd.read_csv(params_avg_path,encoding="utf-8")
        # Get all params from training set
        params_train = params_avg.iloc[train_idx,:].copy()
        # Get avg of them
        params_avg = params_train.reset_index().mean()
        # print(f"params_avg:{params_avg}")

        alpha_avg = params_avg.alpha
        labda_avg = params_avg.labda


        pd_idx = 0
        metric_pd_KL = pd.DataFrame()
        metric_pd_NLL = pd.DataFrame()
        for i in tqdm(test_idx):
                # Get target data
                target_path_i_path = os.path.join(target_path, target_all_path[i])
                target_df = pd.read_csv(target_path_i_path,encoding="utf-8")
                target_df_uniq = pd.DataFrame(np.unique(target_df,axis=0)) # KL-D计算必须有这一步，去掉N值重复的sample
                target = list(target_df.iloc[:,0])  # NLL计算需要这一步


                # Get the auction setting of i_th according to the test_idx
                settings_df = data_key.iloc[i,:]

                metric_pd_KL.loc[pd_idx,col_names_2[0]] = GT2_KL(settings_df,target_df_uniq,alpha_unified,labda_unified)
                metric_pd_KL.loc[pd_idx,col_names_2[1]] = GT2_KL(settings_df,target_df_uniq,alpha_avg,labda_avg)

                metric_pd_NLL.loc[pd_idx,col_names_2[0]] = GT_2_uniq(settings_df,target,alpha_unified,labda_unified)
                metric_pd_NLL.loc[pd_idx,col_names_2[1]] = GT_2_uniq(settings_df,target,alpha_avg,labda_avg)

                pd_idx += 1

        metric_unified_KL.loc[seed,col_names_2[0]] = np.mean(metric_pd_KL.loc[:,col_names_2[0]])
        metric_unified_KL.loc[seed,col_names_2[1]] = np.mean(metric_pd_KL.loc[:,col_names_2[1]])

        metric_unified_NLL.loc[seed,col_names_2[0]] = np.mean(metric_pd_NLL.loc[:,col_names_2[0]])
        metric_unified_NLL.loc[seed,col_names_2[1]] = np.mean(metric_pd_NLL.loc[:,col_names_2[1]])

metric_unified_KL.reset_index(inplace=True)
metric_unified_NLL.reset_index(inplace=True)
print(metric_unified_KL)
print(metric_unified_NLL)

========== seed = 4 ==========


100%|██████████| 255/255 [03:47<00:00,  1.12it/s]


========== seed = 31 ==========


100%|██████████| 255/255 [03:18<00:00,  1.28it/s]


========== seed = 35 ==========


...

========== seed = 508 ==========


100%|██████████| 255/255 [09:49<00:00,  2.31s/it]


========== seed = 512 ==========


100%|██████████| 255/255 [10:18<00:00,  2.43s/it]

   index  GT2(Unified)  GT2(Avg)
0      4      2.286820  4.183363
1     31      2.471957  3.728147
2     35      2.172916  3.860008
3     66      2.366522  3.884415
4    204      4.032703  3.675669
5    407      2.648417  3.943135
6    508      2.311919  3.815558
7    512      2.315638  4.311535
   index  GT2(Unified)  GT2(Avg)
0      4      6.352456  8.254181
1     31      6.527170  7.786769
2     35      6.150913  7.842768
3     66      6.354901  7.877006
4    204      8.026791  7.668687
5    407      6.702764  8.000795
6    508      6.294789  7.802605
7    512      6.442912  8.444024


In [119]:
output_file_KL = output_path + prefix +"GT2_metric_KL.csv"
metric_unified_KL.to_csv(output_file_KL)
output_file_KL

'../../MLP/metric_saved/synthetic_GT2_metric_KL.csv'

In [120]:
output_file_NLL = output_path + prefix +"GT2_metric_NLL.csv"
metric_unified_NLL.to_csv(output_file_NLL)
output_file_NLL

'../../MLP/metric_saved/synthetic_GT2_metric_NLL.csv'

# 4. GT3(Unified)和GT3(Avg)
- 不同于GT1，需要按照seed算


In [263]:
metric_unified_KL = pd.DataFrame()
metric_unified_NLL = pd.DataFrame()

for seed in seed_list:
        setup_seed(seed)
        shuffled_indices = save_data_idx_simplified(len_all)
        train_idx,_,test_idx = get_data_idx(shuffled_indices, train_pct = config.train_pct, vali_pct=config.vali_pct)

        print(f"========== seed = {seed} ==========")

        if not ARTIFICIAL:
                params_unified_path = r"../../data/SA_PT/params_opitim_GT3_unified_seed="+str(seed)+".csv"
                params_avg_path = r"../../data/SA_PT/params_opitim_delta_GT3.csv"
        else:
                params_unified_path = r"../../data/SA_PT/params_artificial_GT3_unified_noise="+str(noise_pct)+ "_seed="+str(seed)+".csv"
                params_avg_path = r"../../data/SA_PT/params_opitim_artificial_GT3_noise_"+str(noise_pct)+".csv"

        # Unified
        params_unified = pd.read_csv(params_unified_path,encoding="utf-8")
        # Get params with the lowest avg_loss
        idx_param = params_unified.avg_loss.idxmin(0)
        params_unified = params_unified.iloc[idx_param,:]
        alpha_unified = params_unified.alpha
        # print(f"params_unified:{params_unified}")

        # Avg
        params_avg = pd.read_csv(params_avg_path,encoding="utf-8")
        # Get all params from training set
        params_train = params_avg.iloc[train_idx,:].copy()
        # Get avg of them
        params_avg = params_train.reset_index().mean()
        print(f"params_avg:{params_avg}")

        alpha_avg = params_avg.alpha

        pd_idx = 0
        metric_pd_KL = pd.DataFrame()
        metric_pd_NLL = pd.DataFrame()
        for i in tqdm(test_idx):
                # Get target data
                target_path_i_path = os.path.join(target_path, target_all_path[i])
                target_df = pd.read_csv(target_path_i_path,encoding="utf-8")
                target_df_uniq = pd.DataFrame(np.unique(target_df,axis=0)) # KL-D计算必须有这一步，去掉N值重复的sample
                target = list(target_df.iloc[:,0])  # NLL计算需要这一步

                # Get the auction setting of i_th according to the test_idx
                settings_df = data_key.iloc[i,:]

                metric_pd_KL.loc[pd_idx,col_names_3[0]] = GT3_KL(settings_df,target_df_uniq,alpha_unified)
                metric_pd_KL.loc[pd_idx,col_names_3[1]] = GT3_KL(settings_df,target_df_uniq,alpha_avg)

                metric_pd_NLL.loc[pd_idx,col_names_3[0]] = GT_3_uniq(settings_df,target,alpha_unified)
                metric_pd_NLL.loc[pd_idx,col_names_3[1]] = GT_3_uniq(settings_df,target,alpha_avg)

                pd_idx += 1

        metric_unified_KL.loc[seed,col_names_3[0]] = np.mean(metric_pd_KL.loc[:,col_names_3[0]])
        metric_unified_KL.loc[seed,col_names_3[1]] = np.mean(metric_pd_KL.loc[:,col_names_3[1]])

        metric_unified_NLL.loc[seed,col_names_3[0]] = np.mean(metric_pd_NLL.loc[:,col_names_3[0]])
        metric_unified_NLL.loc[seed,col_names_3[1]] = np.mean(metric_pd_NLL.loc[:,col_names_3[1]])

metric_unified_KL.reset_index(inplace=True)
metric_unified_NLL.reset_index(inplace=True)
print(metric_unified_KL)
print(metric_unified_NLL)

========== seed = 4 ==========
params_avg:index           6.316361e+02
key_idx         6.316361e+02
alpha           5.512556e-02
delta           1.000000e+00
initial_loss    1.409671e+06
final_loss      1.406431e+06
avg_loss        3.597305e+03
dtype: float64


100%|██████████| 255/255 [07:30<00:00,  1.77s/it]


========== seed = 31 ==========
params_avg:index           6.315073e+02
key_idx         6.315073e+02
alpha           5.400047e-02
delta           1.000000e+00
initial_loss    1.387122e+06
final_loss      1.383908e+06
avg_loss        3.856359e+03
dtype: float64


100%|██████████| 255/255 [07:01<00:00,  1.65s/it]


========== seed = 35 ==========
params_avg:index           6.290370e+02
key_idx         6.290370e+02
alpha           5.650647e-02
delta           1.000000e+00
initial_loss    1.334951e+06
final_loss      1.331663e+06
avg_loss        3.327227e+03
dtype: float64


...

========== seed = 508 ==========
params_avg:index           6.474938e+02
key_idx         6.474938e+02
alpha           5.475814e-02
delta           1.000000e+00
initial_loss    1.476784e+06
final_loss      1.473457e+06
avg_loss        3.773005e+03
dtype: float64


100%|██████████| 255/255 [08:36<00:00,  2.02s/it]


========== seed = 512 ==========
params_avg:index           6.350941e+02
key_idx         6.350941e+02
alpha           5.951197e-02
delta           1.000000e+00
initial_loss    1.192775e+06
final_loss      1.189443e+06
avg_loss        3.272181e+03
dtype: float64


100%|██████████| 255/255 [08:55<00:00,  2.10s/it]

   index  GT3(Unified)   GT3(Avg)
0      4      2.604281   9.481503
1     31      3.170322   8.161458
2     35      2.316207   8.629289
3     66      3.004220   9.256993
4    204      5.608378   8.236234
5    407      4.341136   8.855707
6    508      3.125939   8.703229
7    512      3.141783  10.118436
   index  GT3(Unified)   GT3(Avg)
0      4      6.670822  13.566736
1     31      7.227521  12.232042
2     35      6.294422  12.625513
3     66      6.994390  13.263768
4    204      9.607187  12.242754
5    407      8.399884  12.926124
6    508      7.111192  12.703917
7    512      7.271156  14.266156


In [264]:
metric_unified_KL

,index,GT3(Unified),GT3(Avg)
0,4,2.604281,9.481503
1,31,3.170322,8.161458
2,35,2.316207,8.629289
3,66,3.004220,9.256993
4,204,5.608378,8.236234
5,407,4.341136,8.855707
6,508,3.125939,8.703229
7,512,3.141783,10.118436


In [265]:
output_file_KL = output_path + prefix +"GT3_metric_KL.csv"
metric_unified_KL.to_csv(output_file_KL,index=False)
output_file_KL

'../../MLP/metric_saved/synthetic_GT3_metric_KL.csv'

In [266]:
output_file_NLL = output_path + prefix +"GT3_metric_NLL.csv"
metric_unified_NLL.to_csv(output_file_NLL,index=False)
output_file_NLL

'../../MLP/metric_saved/synthetic_GT3_metric_NLL.csv'

# 5. 拾取GT1在test-set上的metric

In [151]:
metric_KL = pd.DataFrame()
metric_NLL = pd.DataFrame()

for seed in seed_list:
        setup_seed(seed)
        shuffled_indices = save_data_idx_simplified(len_all)
        _,_,test_idx = get_data_idx(shuffled_indices, train_pct = config.train_pct, vali_pct=config.vali_pct)

        KL_metric_path = output_path + prefix +"GT1_metric_KL_all.csv"
        NLL_metric_path = output_path + prefix +"GT1_metric_NLL_all.csv"

        KL_metric = pd.read_csv(KL_metric_path,index_col=0)
        NLL_metric = pd.read_csv(NLL_metric_path,index_col=0)

        metric_KL.loc[seed,col_names_1[0]] = np.mean(KL_metric.loc[test_idx,'GT1'])
        metric_NLL.loc[seed,col_names_1[0]] = np.mean(NLL_metric.loc[test_idx,'GT1'])


In [152]:
output_file_KL = output_path + prefix + "GT1_metric_KL.csv"
metric_KL.to_csv(output_file_KL)
output_file_KL

'../../MLP/metric_saved/real_GT1_metric_KL.csv'

In [153]:
output_file_NLL = output_path + prefix + "GT1_metric_NLL.csv"
metric_NLL.to_csv(output_file_NLL)
output_file_NLL

'../../MLP/metric_saved/real_GT1_metric_NLL.csv'